# [Backtesting.py](https://kernc.github.io/backtesting.py/)

> Backtest trading strategies in Python

backtesting.py는 거래 전략의 실행가능성을 추론하기 위해 과거 데이터에 대한 백테스팅 라이브러리입니다.

물론 과거의 성과가 미래의 결과를 나타내는 것은 아니지만, 다양한 상황에서 입증된 전략은 앞으로도 신뢰할 수 있을 것입니다.

backtesting.py는 [Backtrader](https://www.backtrader.com/)의 비전을 바탕으로 개선되었으며 다른 백테스팅 라이브러리와 비교할 수 없을만큼

가볍고 빠르며 사용자 친화적이고 직관적이고 대화형입니다.  

## 특징

- **외환, 암호화폐, 주식, 선물 시장에 호환됩니다.**
- **기술 지표 라이브러리에 구애받지 않습니다** - (TA-Lib, Tulip 호환가능)
- **시각화 지원**
- **빠르고 편리함** - Pandas, NumPy, Bokeh 기반으로 구축되었습니다.
- **벡터화, 이벤트 기반 백테스팅** - 신호 기반 또는 스트리밍 두 가지 접근 방식의 유연성을 활용하여 전략을 모델링할 수 있습니다.
- **구성된 전략** - 미리 정의된 유틸리티와 스택형 범용 전략 라이브러리가 포함되어있습니다.


참고 문서
- [backtesting.py](https://kernc.github.io/backtesting.py/)
- [Custom Indicators In Backtesting.py - Python Deep Dive](https://youtu.be/xljQpeYQYkI?si=Fa4pMr1ioOxAh5_C)

In [8]:
# Signal Indicator

import numpy as np
from backtesting import Backtest, Strategy
from backtesting.test import GOOG

GOOG['signal'] = np.random.randint(-1, 2, len(GOOG))

print(GOOG.head())

class SignalStrategy(Strategy):
  def init(self):
    pass
  
  def next(self):
    current_signal = self.data.signal[-1]
    if current_signal == 1:
      if not self.position:
        self.buy()
    elif current_signal == -1:
      if self.position:
        self.position.close()
  
bt = Backtest(GOOG, SignalStrategy, cash=1_000_000, commission=.002)

print(bt.run())
bt.plot()

              Open    High     Low   Close    Volume  signal
2004-08-19  100.00  104.06   95.96  100.34  22351900      -1
2004-08-20  101.01  109.08  100.50  108.31  11428600      -1
2004-08-23  110.75  113.48  109.05  109.40   9137200       1
2004-08-24  111.24  111.60  103.57  104.87   7631300       0
2004-08-25  104.96  108.00  103.88  106.00   4598900       0
Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   68.435754
Equity Final [$]                 2569051.0699
Equity Peak [$]                 3574086.17818
Return [%]                         156.905107
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    11.70533
Volatility (Ann.) [%]                29.49869
Sharpe Ratio                         0.396808
Sortino Ratio                        0.707271
Calmar Ratio                         0.233786
Max. Drawdown [%]                  -

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  

GridPlot(id='p1639', ...)

## Position

> 현재 보유 중인 자산 포지션은 Strategy.next() 내에서 Strategy.position으로 사용할 수 있습니다.

### sell vs close

- sell: 새로운 매도 주문을 생성합니다.
- close: 현재 보유 중인 포지션의 일부를 마감합니다.

In [21]:
from backtesting._util import _Data as BacktestData
# Momentum strategy using pandas
from backtesting import Backtest, Strategy

import pandas as pd
import numpy as np


def indicator(data: BacktestData):
  # Data is going to be our OHLCV
  # 현재와 이전 가격의 변화율을 계산합니다.
  return data.Close.s.pct_change(periods=7) * 100 

class MomentumStrategy(Strategy):
  ptc_change: np.ndarray

  def init(self):
    self.pct_change: np.ndarray = self.I(indicator, self.data)
    
  def next(self):
    # 현재 변화율을 가져옵니다.
    change = self.pct_change[-1] 
      
    # 현재 포지션이 있으면, 변화율이 음수이면 포지션을 닫습니다.
    if self.position:
      if change < 0:
        self.position.close()
    # 현재 포지션이 없으면, 변화율이 5보다 크고 이전 변화율이 5보다 크면 매수합니다.
    else:
      if change > 5 and self.pct_change[-2] > 5:
        self.buy()

bt = Backtest(GOOG, MomentumStrategy, cash=1_000_000, commission=.002)

print(bt.run())
bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    28.67784
Equity Final [$]                 3259494.1284
Equity Peak [$]                  3296463.3784
Return [%]                         225.949413
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   14.868772
Volatility (Ann.) [%]               20.531546
Sharpe Ratio                         0.724192
Sortino Ratio                        1.414668
Calmar Ratio                         0.582949
Max. Drawdown [%]                   -25.50611
Avg. Drawdown [%]                   -4.610434
Max. Drawdown Duration      705 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                                   51
Win Rate [%]                        43.137255
Best Trade [%]                      54.295245
Worst Trade [%]                     -8.950836
Avg. Trade [%]                    

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  

GridPlot(id='p3686', ...)

In [22]:
import json
from backtesting.lib import SignalStrategy, TrailingStrategy

with open('../data/ohlcv/twelvedata/time_series_233740_1day.json') as f:
    data = json.load(f)

# Convert data to DataFrame
twelve_data_types = {'datetime': 'datetime64[ns]', 'open': 'float64', 'high': 'float64', 'low': 'float64',
                     'close': 'float64'}

df = (pd.DataFrame(data['values'])
      .drop_duplicates()
      .astype(twelve_data_types)
      .sort_values('datetime')
      .set_index('datetime'))

# Rename columns to match backtesting.py requirements
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

class VolatilityStrategy(Strategy):
  def init(self):
    # 묙표가 = 금일시가 + (전일고가 - 전일저가) * 0.5
    self.target_price = self.I(lambda x: x.Open + (x.High - x.Low) * 0.5, self.data) 
    print(self.target_price)

  def next(self):
    # 고가가 목표 가격보다 높으면 매수
    if self.data.High > self.target_price:
      self.buy()
    # 다음날 시가 매도
    elif self.data.High[-1] > self.target_price[-1]:
      self.sell()
    
      
bt = Backtest(df, VolatilityStrategy, cash=1_000_000, commission=.002)

print(bt.run())
bt.plot()

[12180.  11240.  11677.5 ... 10617.5 10600.  10340. ]
Start                     2016-01-04 00:00:00
End                       2024-05-24 00:00:00
Duration                   3063 days 00:00:00
Exposure Time [%]                   99.903241
Equity Final [$]                    885245.52
Equity Peak [$]                     2453146.3
Return [%]                         -11.475448
Buy & Hold Return [%]               -8.235824
Return (Ann.) [%]                   -1.490798
Volatility (Ann.) [%]               56.669517
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.883677
Avg. Drawdown [%]                  -20.828785
Max. Drawdown Duration     2320 days 00:00:00
Avg. Drawdown Duration      609 days 00:00:00
# Trades                                    2
Win Rate [%]                              0.0
Best Trade [%]                      -8.501343
Worst Trade [%]           

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  

GridPlot(id='p4031', ...)

In [15]:
from backtesting.lib import resample_apply
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting._util import _Data as BacktestData
import numpy as np

# momentum2
# https://youtu.be/9m987swadQU?si=MlrEVg2EXxgvdRsS

class MomentumStrategy(Strategy):
  small_threshold = 0
  large_threshold = 3
  
  def momentum(self, data: BacktestData, period = 7):
    return data.pct_change(periods = period).to_numpy() * 100
  
  def init(self):
    self.pct_change_long = resample_apply('2h', self.momentum, self.data.Close.s)
    self.pct_change_short = resample_apply('30T', self.momentum, self.data.Close.s)
      
  def next(self):
    change_long = self.pct_change_long[-1]
    change_short = self.pct_change_short[-1]
      
    if change_long > 5 and change_short > 5:
      self.buy()
    elif change_long < 0 and change_short < 0:
      self.position.close()
          
      if self.position:
        # check whether we should close the position
        if self.position.is_long and change_short < self.small_threshold:
          self.position.close()
        elif self.position.is_short and change_short > -self.small_threshold:
          self.position.close()
      else:
        # check whether we should go long / short
        if change_long > self.large_threshold and change_short > self.small_threshold:
          self.buy()
        elif change_long < -self.large_threshold and change_short < -self.small_threshold:
          self.sell()
                  
                  
bt = Backtest(GOOG, MomentumStrategy, cash=10_000_000, commission=.002)
stats = bt.optimize(
  small_threshold = list(np.arange(0,1,0.1)),
  large_threshold = list(np.arange(1,3,0.2)),
  maximize='Equity Final [$]',
)
bt.run()
bt.plot()

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/lib.py:294: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled = series.resample(rule, label='right').agg(agg).dropna()
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/lib.py:294: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled = series.resample(rule, label='right').agg(agg).dropna()
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/lib.py:294: FutureWarning: 'T' is deprecated and will be removed in a future version, pleas

GridPlot(id='p1821', ...)

In [16]:
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   69.832402
Equity Final [$]               11393684.48546
Equity Peak [$]                 20025726.9416
Return [%]                          13.936845
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    1.542477
Volatility (Ann.) [%]               27.227997
Sharpe Ratio                          0.05665
Sortino Ratio                         0.08331
Calmar Ratio                         0.028249
Max. Drawdown [%]                  -54.602941
Avg. Drawdown [%]                   -8.775229
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration      182 days 00:00:00
# Trades                                  409
Win Rate [%]                        46.210269
Best Trade [%]                      57.156743
Worst Trade [%]                    -19.562846
Avg. Trade [%]                    

In [17]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-101018,10,11,98.99162,100.95,-1.978316e+05,-0.019783,2004-09-02,2004-09-03,1 days
1,-97236,12,13,100.80798,100.74,6.610103e+03,0.000674,2004-09-07,2004-09-08,1 days
2,-95859,14,15,102.32494,101.60,6.949202e+04,0.007085,2004-09-09,2004-09-10,1 days
3,89169,18,57,110.78112,174.10,5.646081e+06,0.571567,2004-09-15,2004-11-09,55 days
4,-91143,58,59,170.32866,169.13,1.092495e+05,0.007037,2004-11-10,2004-11-11,1 days
...,...,...,...,...,...,...,...,...,...,...
404,-15594,2107,2108,717.98116,724.93,-1.083602e+05,-0.009678,2013-01-02,2013-01-03,1 days
405,14914,2114,2117,743.48400,722.40,-3.144468e+05,-0.028358,2013-01-11,2013-01-16,5 days
406,-15197,2119,2120,708.93928,704.66,6.503222e+04,0.006036,2013-01-18,2013-01-22,4 days
407,-14756,2121,2122,734.51802,741.24,-9.918954e+04,-0.009152,2013-01-23,2013-01-24,1 days


In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now 

GridPlot(id='p1325', ...)

In [7]:
import json
import logging
import pandas as pd
import numpy as np
import dataclasses as dc

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting._util import _Data as BacktestData
from backtesting.test import GOOG, SMA


@dc.dataclass
class VolatilityBreakoutResult:
    """ Result of volatility breakout strategy.

    Attributes:
        cagr (float): Compound annual growth rate.
        mdd (float): Maximum drawdown.
        simple_hpr (float): Simple holding period return.
        hpr (float): Holding period return.
    """
    cagr: float
    mdd: float
    simple_hpr: float
    hpr: float


def calculate_hpr(first_close: float, last_close: float) -> float:
    """ Calculate holding period return.

    Args:
        first_close (float): First close price.
        last_close (float): Last close price.

    Returns:
        float: Holding period return.
    """
    return (last_close - first_close) / first_close


def calculate_noise_ratio(open: float, high: float, low: float, close: float) -> float:
    """ Calculate noise ratio.

    Args:
        open (float): Open price.
        high (float): High price.
        low (float): Low price.
        close (float): Close price.

    Returns:
        float: Noise ratio.
    """

    return 1 - abs(open - close) / (high - low) if high - low != 0 else 0


def volatility_breakout(
        df: pd.DataFrame,
        noise_ratio_period: int = 13,
        fee: float = 0.1,
        moving_average_period: int = 5,
) -> VolatilityBreakoutResult:
    """ Volatility breakout strategy.

    Args:
        df (pd.DataFrame): DataFrame with open, high, low, close, and volume data.
        noise_ratio_period (int): Noise ratio period days. (Default is 13 days).
        fee: Slippage and commission fee. (Default is 0.1%).
        trailing_stop: Trailing stop loss. (Default is 5%).

    Returns:
        VolatilityBreakoutResult: Result of volatility breakout strategy.
    """

    # Calculate holding period returns
    simple_hpr = calculate_hpr(df['close'].iloc[0], df['close'].iloc[-1])

    # Add Noise Ratio and Average Noise Ratio to DataFrame
    df['noise_ratio'] = df.apply(lambda x: calculate_noise_ratio(x['open'], x['high'], x['low'], x['close']), axis=1)
    df['average_noise_ratio'] = df['noise_ratio'].rolling(window=noise_ratio_period).mean()

    # Add target price
    df['range'] = (df['high'] - df['low']).shift(1)
    df['next_open'] = df['open'].shift(-1)
    df['target_price'] = df['open'] + df['range'] * df['average_noise_ratio']

    # Add market timing
    df['md'] = df['close'].rolling(window=moving_average_period).mean()

    # Add buy signal
    df['buy_signal'] = np.where((df['high'] > df['target_price']) & (df['high'] > df['md']), True, False)

    # Calculate Rate of Return
    df['ror'] = np.where(df['buy_signal'], df['next_open'] / df['target_price'] - fee / 100, 1)

    # Holding period return
    df['hpr'] = df['ror'].cumprod()

    # Maximum drawdown
    # (high - low) / high
    df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax()

    df.dropna(inplace=True)

    # Calculate CAGR and MDD
    # CAGR = (Ending Value / Beginning Value) ^ (1 / Number of Years) - 1
    cagr = df['hpr'][-1] ** (252 / len(df)) - 1
    mdd = df['dd'].max()
    hpr = df['hpr'][-1]

    return VolatilityBreakoutResult(
        cagr=cagr,
        mdd=mdd,
        simple_hpr=simple_hpr,
        hpr=hpr
    )

# logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# stream_handler = logging.StreamHandler() ## 스트림 핸들러 생성
# logger.addHandler(stream_handler) ## 핸들러 등록

def calculate_noise_ratio(open: float, high: float, low: float, close: float) -> float:
    """ Calculate noise ratio.

    Args:
        open (float): Open price.
        high (float): High price.
        low (float): Low price.
        close (float): Close price.

    Returns:
        float: Noise ratio.
    """

    return 1 - abs(open - close) / (high - low) if high - low != 0 else 0


class VolatilityBreakoutStrategy(Strategy):
    noise_ratio_period = 20
    moving_average_period = 5

    def init(self):
        # logger.info('init')
        # 이동평균선 추가
        pass

    def next(self):
        # 날짜와 데이터 출력
        if not self.position and self.data['buy_signal'][-1]:
            # logger.info(self.position)
            # logger.info(f'buy {self.data.index[-1]}: {self.data["Close"][-1]} > {self.data["target_price"][-1]}')
            self.buy(limit=self.data['target_price'][-1])
            self.position.close()
        # 다음날 시가에 매도
        elif self.position:
            # logger.info(self.position)
            # logger.info(f'sell {self.data.index[-1]}: {self.data["Open"][-1]}')
            self.position.close()
            # logger.info(self.position)

# Load data
with open('time_series_233740_1day.json') as f:
    data = json.load(f)

# Convert data to DataFrame
twelve_data_types = {'datetime': 'datetime64[ns]', 'open': 'float64', 'high': 'float64', 'low': 'float64',
                     'close': 'float64'}
df = (pd.DataFrame(data['values'])
      .drop_duplicates()
      .astype(twelve_data_types)
      .sort_values('datetime')
      .set_index('datetime'))

# Calculate volatility breakout strategy
result = volatility_breakout(df, noise_ratio_period=20, fee=0.0)

# convert columes for backtesting
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'noise_ratio', 'average_noise_ratio', 'range', 'next_open', 'target_price', 'md', 'buy_signal', 'ror', 'hpr', 'dd']

# Run backtest
bt = Backtest(df, VolatilityBreakoutStrategy, cash=1_000_000_000, commission=0.0015)
stats = bt.run()
bt.plot()

print(stats)

stats._trades

/var/folders/zs/1ztn5m_s7q78gg656_rywdw80000gn/T/ipykernel_2578/2442491623.py:108: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cagr = df['hpr'][-1] ** (252 / len(df)) - 1
/var/folders/zs/1ztn5m_s7q78gg656_rywdw80000gn/T/ipykernel_2578/2442491623.py:110: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hpr = df['hpr'][-1]
/Users/corn/Desktop/code/quant-strategy/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/corn/Desktop/code/quant-str

Start                     2016-01-29 00:00:00
End                       2024-05-23 00:00:00
Duration                   3037 days 00:00:00
Exposure Time [%]                   39.863214
Equity Final [$]            3645258992.080997
Equity Peak [$]             4004792891.378716
Return [%]                         264.525899
Buy & Hold Return [%]              -10.237581
Return (Ann.) [%]                   17.463781
Volatility (Ann.) [%]               25.703117
Sharpe Ratio                         0.679442
Sortino Ratio                        1.225189
Calmar Ratio                         0.597468
Max. Drawdown [%]                  -29.229647
Avg. Drawdown [%]                   -4.697935
Max. Drawdown Duration      927 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                  418
Win Rate [%]                        56.220096
Best Trade [%]                      14.541598
Worst Trade [%]                     -9.915594
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,60994,461,462,16395.000000,16065.0,-2.012802e+07,-0.020128,2017-12-21,2017-12-22,1 days
1,49956,468,469,19614.340231,20000.0,1.926602e+07,0.019662,2018-01-04,2018-01-05,1 days
2,45711,472,473,21857.310627,22170.0,1.429334e+07,0.014306,2018-01-10,2018-01-11,1 days
3,36852,476,477,27500.000000,28560.0,3.906312e+07,0.038545,2018-01-16,2018-01-17,1 days
4,42762,480,481,24612.378997,25500.0,3.795645e+07,0.036064,2018-01-22,2018-01-23,1 days
...,...,...,...,...,...,...,...,...,...,...
413,308771,2019,2020,11240.000000,11575.0,1.034383e+08,0.029804,2024-04-11,2024-04-12,1 days
414,321116,2022,2023,11130.000000,11055.0,-2.408370e+07,-0.006739,2024-04-16,2024-04-17,1 days
415,322606,2025,2026,11003.932719,11270.0,8.583490e+07,0.024179,2024-04-19,2024-04-22,3 days
416,309427,2030,2031,11750.000000,11755.0,1.547135e+06,0.000426,2024-04-26,2024-04-29,3 days
